In [76]:
import ujson as json
from collections import defaultdict, Counter
from datetime import datetime
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF
import numpy as np

In [2]:
with open('./words_preprocessed.json') as f:
    data = json.load(f)

In [210]:
stop_words = set(['기후', '하다', '있다', '되다', '돼다', '위해', '이다', '않다', '따르다',
                  '대한', '통한', '없다', '오다', '보다', '아니다', '통해', '같다', '밝히다',
                  '되어다', '가다', '대해', '이렇다', '어떻다', '가지', '나타나다', '경우',
                  '이번', '지난', '우리', '들다', '또한'])
word_count = Counter()
for article in data:
    if len(article['tokens']) > 0:
        word_count.update([w for w in article['tokens'] if w not in stop_words and len(w) > 1])

total_words = 20000
word_count = sorted([(w, c) for w, c in word_count.items()], key=lambda x: x[1], reverse=True)
voca = sorted([w for w, c in word_count[:total_words]])
word2id = {w: i for i, w in enumerate(voca)}

In [211]:
data_ = []
dates = set()
min_date = datetime(2017, 1, 1).date()
for article in data:
    words = [w for w in article['tokens'] if w in word2id]
    date = datetime.fromtimestamp(article['date']).date()
    if len(words) >= 10 and date >= min_date:
        article_ = {
            'title': article['title'],
            'date': date,
            'text': article['text'],
            'tokens': words
        }
        data_.append(article_)
        dates.add(date)
        
dates = sorted(dates)
date2id = {d: i for i, d in enumerate(dates)}

In [212]:
tdm = np.zeros((len(data_), len(voca)), dtype=np.float32)
for i, article in enumerate(data_):
    for word in article['tokens']:
        if word in word2id:
            tdm[i, word2id[word]] += 1

tdm = TfidfTransformer().fit_transform(tdm)

In [213]:
K = 10
nmf = NMF(n_components=K, alpha=0.1, max_iter=500)
W = nmf.fit_transform(tdm)
H = nmf.components_

for k in range(K):
    print(f"{k}th topic")
    keyword_indices = H[k, :].argsort()[::-1][:25]
    for index in keyword_indices:
        print(voca[index], end=' ')
    print()

0th topic
협력 산업 정책 국제 기술 한국 기업 분야 포럼 발전 개발 연구 회의 세계 사회 환경 경제 장관 해양 개최 미래 과장 가능 글로벌 관리 
1th topic
대통령 트럼프 미국 협정 파리 탈퇴 정상 중국 마크 도널드 프랑스 총리 오바마 독일 북한 회의 메르켈 협약 유엔 기후변화 밉다 세계 행정부 백악관 정상회담 
2th topic
에너지 태양광 전력 신재생에너지 발전 전환 원전 산업 재생에너지 석탄 효율 기술 정책 발전소 자립 재생 절약 공단 설비 한국 생산 전기 친환경 풍력 소비 
3th topic
농업 기술 재배 농가 농업인 생산 소득 작물 농촌 센터 농산물 사업 품종 품질 해충 육성 지원 사과 교육 방제 과수 시범 개발 보급 영농 
4th topic
폭염 기온 피해 발생 재난 상승 기록 기상청 지구 영향 인하다 이상 때문 온도 증가 여름 가장 기상 지역 태풍 높다 많다 평균 최고 크다 
5th topic
사업 미세먼지 지원 추진 차량 설치 시설 시민 서울시 보급 조성 계획 도시 예산 환경 태양광 개선 전기차 보조금 신청 자동차 노후 동차 대다 지역 
6th topic
온실가스 감축 배출 탄소 기후변화 목표 실천 포인트 할당 배출량 줄이다 부문 파리 기업 거래 배출권거래제 이산화탄소 협정 녹색 탄소배출권 캠페인 협약 생활 친환경 계획 
7th topic
산림 산불 산림청 나무 방제 해충 발생 진화 자원 피해 가꾸다 예방 유림 조림 조성 방지 심기 목재 헬기 임업 수목원 산사태 지역 돌발 수종 
8th topic
교육 환경 체험 행사 실천 프로그램 시민 생태 학교 학생 생활 진행 참여 운영 기후변화 대상 센터 캠페인 다양하다 활동 청소년 녹색 축제 문화 어린이 
9th topic
가뭄 용수 저수지 농어촌 공사 대책 관리 공급 지역 안전 확보 수자원 사장 점검 피해 농업 사업 대비 재해 시설 추진 재난 평년 충남 급수 


In [214]:
tdm_date = np.zeros((len(dates), len(voca)), dtype=np.float32)
for article in data_:
    for word in article['tokens']:
        if word in word2id:
            tdm_date[date2id[article['date']], word2id[word]] += 1

tdm_date = TfidfTransformer().fit_transform(tdm_date)

In [225]:
K = 20
nmf = NMF(n_components=K, alpha=0.1, max_iter=500)
W = nmf.fit_transform(tdm_date)
H = nmf.components_

for k in range(K):
    print(f"{k}th topic")
    keyword_indices = H[k, :].argsort()[::-1][:25]
    for index in keyword_indices:
        print(voca[index], end=' ')
    print()

0th topic
기술 한국 협력 산업 개발 기후변화 지역 세계 기업 발전 환경 분야 도시 국제 관리 사회 시장 연구 대응 정책 지속 미래 평가 경제 진행 
1th topic
미국 트럼프 대통령 파리 협정 탈퇴 기후변화 중국 세계 정부 도널드 협약 가뭄 환경 오바마 문제 늘다 국가 에너지 마크 정책 총리 결정 시장 백악관 
2th topic
정책 관실 사업 환경 자치행정 교육 본부 산업 담당 건설 계획 복지 문화 행정 지원 경제 기획 기술 보건 트럼프 상수도 승진 도시 다보스 안전 
3th topic
폭염 피해 지역 기온 발생 누진 기후변화 더위 대책 여름 기록 이상 관리 온도 무더위 인하다 기술 전기요금 농업 에너지 최고 높다 많다 재난 증가 
4th topic
에너지 사업 발전 기술 정책 산업 지역 계획 태양광 한국 신재생에너지 환경 전환 추진 전력 확대 개발 도시 생산 석탄 효율 기후변화 시장 정부 분야 
5th topic
사업 지역 추진 관리 계획 시설 가뭄 환경 지원 기후변화 용수 설치 정부 대응 대책 개발 공급 도시 시민 확보 산업 올해 미세먼지 발생 발전 
6th topic
대통령 정상 회의 협력 정상회담 프랑스 양국 북한 총리 독일 문재인 한국 메르켈 유럽 문제 함부르크 덴마크 평화 순방 방문 한반도 교황 비핵화 마크 참석 
7th topic
환경 기후변화 지역 교육 행사 지구 에너지 체험 시민 온실가스 진행 실천 참여 다양하다 운영 생활 농업 소등 개최 계획 센터 지원 탄소 대응 녹색 
8th topic
원전 정부 정책 에너지 호기 발전 신고리 건설 문제 원자력 대통령 한국 전력 지역 시민 중단 사회 국민 발생 안전 세계 석탄 사고 받다 환경 
9th topic
과장 정책 예산 관리 지원 승진 전보 총괄 교도소 기획 교정청 담당 산업 보안과장 총무 보호관찰 대리 교육 환경 부장 지역 국장 직무 파견 기술 
10th topic
조간 차관 회의 장관 개최 뉴트리아 부총리 한국 석간 중국 국무회의 우렁이 미국 위원장 동향 크다 연어 결과 기후변화 지

In [226]:
from pyecharts import Line

line = Line(width=960, height=600)
for k in range(K):
    line.add(f'{k}th topic', dates, W[:, k],
             is_symbol_show=False, is_smooth=True,
             is_fill=True, is_stack=True)

line

In [263]:
from pyecharts import Line

line = Line(width=960, height=600)
line.add(f'폭염', dates, np.array(tdm_date[:, word2id['폭염']].todense()).reshape(-1).tolist(),
         is_symbol_show=False, is_smooth=True,
         is_fill=True, is_stack=True)

line

In [260]:
len(np.array(tdm[:, word2id['폭염']].todense()).reshape(-1).tolist())

94222

In [218]:
[voca[i] for i in np.array(tdm_date.sum(axis=0)).reshape(-1).argsort()[::-1][:30]]

['사업',
 '지역',
 '에너지',
 '환경',
 '기후변화',
 '기술',
 '한국',
 '정책',
 '계획',
 '산업',
 '세계',
 '미국',
 '지원',
 '정부',
 '관리',
 '받다',
 '추진',
 '발전',
 '늘다',
 '대통령',
 '개발',
 '농업',
 '많다',
 '크다',
 '대응',
 '도시',
 '발생',
 '문제',
 '생산',
 '기업']

In [229]:
H[:, word2id['탄소배출권']]

array([0.        , 0.00307481, 0.        , 0.00602729, 0.        ,
       0.        , 0.        , 0.        , 0.00239003, 0.        ,
       0.        , 0.        , 0.        , 0.01087988, 0.00108531,
       0.        , 0.        , 0.        , 0.        , 0.11767975])

In [237]:
tdm[:, word2id['안철수']].nonzero()[0][0]

array([   83,   201,   268,   672,  1190,  1727,  1771,  2013,  2172,
        2203,  3071,  3651,  3793,  4069,  4177,  4203,  4609,  5101,
        5121,  5191,  7103,  7338,  7668,  7697,  7726,  8067,  8716,
        9011,  9342,  9671,  9886,  9899,  9949,  9955, 10297, 11431,
       12115, 12185, 12635, 12808, 12838, 13080, 13411, 13421, 13483,
       13504, 13627, 13871, 13914, 14010, 14732, 14808, 14924, 15686,
       15732, 15875, 16325, 16547, 16845, 17092, 17528, 17719, 18812,
       18994, 19276, 20533, 21215, 21761, 21798, 21831, 22039, 22337,
       22828, 23144, 23604, 24943, 25631, 25746, 25774, 26280, 26523,
       27406, 28453, 28980, 29123, 30537, 30839, 31424, 31451, 31467,
       31579, 31944, 32664, 34003, 34286, 34714, 35062, 35829, 35871,
       36152, 36168, 36310, 36579, 36706, 36799, 37452, 38176, 38305,
       38440, 38680, 39566, 40929, 41532, 41971, 42348, 42497, 42688,
       42930, 43524, 44027, 44100, 44273, 44564, 44568, 44594, 44746,
       44969, 45058,

In [245]:
data_[tdm[:, word2id['연어']].nonzero()[0][4]]

{'title': '태화강 회귀 연어 고작 123마리-국민일보',
 'date': datetime.date(2017, 3, 27),
 'text': '▶"국민께 사죄 안 하면 비리 폭로한다" 국정농단 예언한 박관천의 경고 ▶마티즈 탄 미모의 BJ, 운전 도중 생방송하다 즉사 ▶‘그날의 단서’ 타각지시기·로그기록 확보해야 ▶“아직 나무는 위험해” 판다 어미의 모성, 새끼의 반항 (영상) ▶이게 현실… 밤에 본 미세먼지는 더 충격적이다 (영상) ▶"탕수육 시켜주겠다" 특검에 이재용 부회장의 대답 ▶질질 끌고가 발로 퍽… 강아지유치원 옥상에서 촬영된 영상 ▶세월호에 뚫린 구멍 ‘천공’이 처음 나타났다 지난해 태화강으로 돌아온 연어가 8년 만에 가장 적은 것으로 조사됐다. 울산시는 지난해 태화강 회귀 연어가 총 123마리에 그쳤다고 27일 밝혔다.시에 따르면 2003년부터 회귀하기 시작한 태화강 연어는 2003년부터 2008년까지 6년 동안 307마리가 돌아온 것을 비롯해 2009년 614마리, 2010년 716마리, 2011년 271마리, 2012년 592마리 등 많은 연어가 돌아왔다. 2013년과 2014년에는 각각 1788마리와 1827마리가 돌아와 최고점을 찍었다.그러나 2015년에는 578마리로 줄었고, 2016년에는 123마리로 급감했다. 연어가 돌아오는 시기는 10월 중순부터 11월 중순에 집중되는데 지난해 숫자가 급감한 것은 기후의 영향과 10월 초 울산을 휩쓸고 간 태풍 차바의 영향이 큰 것으로 보인다.시 관계자는 “태풍으로 태화강 하천이 온통 흙탕물이 되고 연어가 올라오는 강바닥 길목 등지에는 자갈과 돌이 마구 쌓이는 등 지형변화가 생긴 것이 회귀에 큰 장애물이 됐을 것”이라고 말했다. 바닷물 수온이 올라가는 엘니뇨 같은 기후도 연어 회귀에 영향을 미칠 수 있는 것으로 알려져 있어 이밖에도 여러 가지 환경 요인이 작용했을 것으로 추정된다.울산=조원일 기자 wcho@kmib.co.kr',
 'tokens': ['국민',
  '비리',
  '폭로',
 